In [ ]:
import os, sys,pytz
import django
from datetime import datetime
sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps

django.setup()

tz = pytz.timezone('Africa/Gaborone')

Remove mother from Quaterly schedule and put them on the follow up schedule to match the child 

In [2]:
from flourish_caregiver.models import CaregiverOffSchedule
from flourish_child.models import ChildOffSchedule
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from edc_visit_schedule.models import SubjectScheduleHistory
from edc_visit_schedule.constants import ON_SCHEDULE,OFF_SCHEDULE


child_offschedule = ChildOffSchedule.objects.get(subject_identifier='B142-040990669-9-60',schedule_name='child_a_quart_schedule1')
offschedule_date = child_offschedule.offschedule_datetime.astimezone(tz)

subject_identifier = 'B142-040990669-9'
schedule_name = 'a_quarterly2_schedule1'


_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model='flourish_caregiver.onschedulecohortaquarterly',name=schedule_name
)

schedule.take_off_schedule(
    subject_identifier=subject_identifier,
    offschedule_datetime=offschedule_date,
    schedule_name=schedule_name
)

print("Taken of",schedule)


try:    
    subject_history = SubjectScheduleHistory.objects.get(subject_identifier=subject_identifier,schedule_name=schedule_name)
except SubjectScheduleHistory.DoesNotExist:
    pass
else:
    subject_history.offschedule_datetime =offschedule_date
    subject_history.schedule_status = OFF_SCHEDULE

    print(f'Participant {subject_identifier} is offschduele {schedule_name}')

Taken of a_quarterly2_schedule1
Participant B142-040990669-9 is offschduele a_quarterly2_schedule1


In [3]:
from flourish_caregiver.models import MaternalVisit

new_schedule_name = 'a_fu2_schedule1'
maternal_visit = MaternalVisit.objects.get(subject_identifier=subject_identifier,visit_code='2000M')
report_datetime = maternal_visit.report_datetime
report_datetime = report_datetime.astimezone(tz)

_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model='flourish_caregiver.onschedulecohortafu',name=new_schedule_name
)
schedule.put_on_schedule(
    subject_identifier=subject_identifier,onschedule_datetime=offschedule_date,
    schedule_name=new_schedule_name,base_appt_datetime=offschedule_date
)
print("Schedule created for",schedule)

Schedule created for a_fu2_schedule1


In [5]:
from flourish_caregiver.models import OnScheduleCohortAFU
onschedule_mother = OnScheduleCohortAFU.objects.get(subject_identifier=subject_identifier,child_subject_identifier='')
onschedule_mother.child_subject_identifier= child_offschedule.subject_identifier
onschedule_mother.save()

print("Linked with child")

Linked with child
